In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

headers = {
    "User-Agent": "Mozilla/5.0"
}

# URL base ordenada por fecha (más recientes)
base_url = "https://www.remax.com.ar/listings/buy?page={}&pageSize=24&sort=-createdAt&in:operationId=1&in:eStageId=0,1,2,3,4&locations=in:::458@%3Cb%3ESanta%3C%2Fb%3E%20%3Cb%3EFe%3C%2Fb%3E%20Capital::::&landingPath=&filterCount=0&viewMode=listViewMode"

# Número de páginas a recorrer (ajustable)
total_paginas = 100

data = []

for pagina in range(total_paginas):
    print(f"Procesando página {pagina + 1}")
    url = base_url.format(pagina)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.select('.card-remax__container')

    for card in cards:
        nombre = card.select_one('.card__description')
        precio = card.select_one('.card__price')
        direccion = card.select_one('.card__address')
        m2 = card.select_one('.feature--m2total span')
        ambientes = card.select_one('.feature--ambientes span')
        baños = card.select_one('.feature--bathroom span')

        data.append({
            "Nombre": nombre.get_text(strip=True) if nombre else "",
            "Precio USD": precio.get_text(strip=True).replace("USD", "").strip() if precio else "",
            "Dirección": direccion.get_text(strip=True) if direccion else "",
            "M2 Totales": m2.get_text(strip=True).replace(",", ".") if m2 else "",
            "Ambientes": ambientes.get_text(strip=True) if ambientes else "",
            "Baños": baños.get_text(strip=True) if baños else ""
        })

    time.sleep(1.5)  # Espera entre requests

df = pd.DataFrame(data)
df.to_csv("propiedades_bruto.csv", index=False, encoding='utf-8')

Procesando página 1
Procesando página 2
Procesando página 3
Procesando página 4
Procesando página 5
Procesando página 6
Procesando página 7
Procesando página 8
Procesando página 9
Procesando página 10
Procesando página 11
Procesando página 12
Procesando página 13
Procesando página 14
Procesando página 15
Procesando página 16
Procesando página 17
Procesando página 18
Procesando página 19
Procesando página 20
Procesando página 21
Procesando página 22
Procesando página 23
Procesando página 24
Procesando página 25
Procesando página 26
Procesando página 27
Procesando página 28
Procesando página 29
Procesando página 30
Procesando página 31
Procesando página 32
Procesando página 33
Procesando página 34
Procesando página 35
Procesando página 36
Procesando página 37
Procesando página 38
Procesando página 39
Procesando página 40
Procesando página 41
Procesando página 42
Procesando página 43
Procesando página 44
Procesando página 45
Procesando página 46
Procesando página 47
Procesando página 48
P

In [3]:
import pandas as pd

# Leer el CSV
df = pd.read_csv("propiedades_bruto.csv")

# Filtrar precios no válidos
df = df[~df['Precio USD'].astype(str).str.contains('ARS|Solicitar', na=False)]

# Eliminar puntos y comas, convertir a número
df['Precio USD'] = df['Precio USD'].astype(str).str.replace('.', '', regex=False)
df['Precio USD'] = pd.to_numeric(df['Precio USD'], errors='coerce')

# Limpiar y convertir m2
df['M2 Totales'] = df['M2 Totales'].astype(str).str.replace(',', '.', regex=False)
df['M2 Totales'] = pd.to_numeric(df['M2 Totales'], errors='coerce')

# Eliminar filas con valores faltantes
df = df.dropna(subset=['Precio USD', 'M2 Totales'])

# Eliminar duplicados
df = df.drop_duplicates(subset=['Dirección', 'Precio USD', 'M2 Totales'])

# Mapear patrones por tipo incluyendo sinónimos y abreviaturas
tipos = {
    'Departamento': r'\b(departamento|dpto|dto|dept|depto)\b',
    'Casa': r'\bcasa\b',
    'Terreno': r'\bterreno\b',
    'Salón': r'\bsal[oó]n\b',
    'Galpón': r'\bgalp[oó]n\b',
    'Cochera': r'\bcochera\b',
    'Local': r'\blocal\b',
    'Oficina': r'\boficina\b',
    'PH': r'\bPH\b',
    'Campo': r'\bcampo\b',
    'Quinta': r'\bquinta\b'
}

def detectar_tipo(nombre):
    nombre = str(nombre).lower()
    for tipo, patron in tipos.items():
        if pd.notna(nombre) and pd.Series(nombre).str.contains(patron, regex=True).any():
            return tipo
    return "Otro"

df['Tipo'] = df['Nombre'].apply(detectar_tipo)

df.to_csv("propiedades_limpias_tipo.csv", index=False, encoding='utf-8')

C:\Users\estan\AppData\Local\Temp\ipykernel_6640\1577009241.py:41: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if pd.notna(nombre) and pd.Series(nombre).str.contains(patron, regex=True).any():
